In [7]:
from custom_dataset import Dataset_Assay, custom_collate
from models import *
from saveAndLoad import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
# for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data_emb = labeled_data[17]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)

nlabels = len(data_df_emb['int_label'].unique())
print('n labels:',nlabels)
device = 'cuda:1'

mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
assays = pickleLoad('../aa/assays.pkl')

dataset = Dataset_Assay(data_df_emb, mut_embeddings, ref_embeddings, tumors, assays, device)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 100, collate=custom_collate)

data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
n labels: 53
loading data from ../aa/tumors.pkl
loading data from ../aa/assays.pkl


In [8]:
## HYPERPARAMS
num_epochs = 10
learning_rate = 0.001

In [9]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False

config_att = Config_Att()
config_att.n_labels = nlabels

model = Classifier(config_att)
model.to(device)

num_epochs = 15
train_assay(model,num_epochs,train_loader,test_loader)

TRAINING:   0%|          | 0/904 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [391] at entry 0 and [453] at entry 1

In [ ]:
## BATCH SIZE > 1 IS CAUSING NAN LOSS